# Load the data

In [ ]:
using DrWatson
@quickactivate :FlowComplexity

using JLD2

fn = joinpath("./data", "data.jld2")
@load fn sim_array

# Compute population statistics

In [ ]:
using DataFrames, Statistics

# get the length of the array
nsim = length(sim_array)

# get the first simulation of the array
first_sim = sim_array[1]

# define the array that will hold our population time series
max_t = Int(first_sim.total_time)
nspecies = 10
pop_array = zeros(Int, nsim, max_t, nspecies)

for i in 1:nsim
    sim = sim_array[i]
    time_series_df = FlowComplexity.convert_timeseries_to_tidy_df(sim.time_evolution)
        for j in 1:max_t
            for k in 1:nspecies
                filtered_time_series_df = filter(row -> row[:time] == j && row[:variable] == string(k), time_series_df) # fucking variable encoded as string
                pop_array[i, j, k] = filtered_time_series_df[1, :value]
            end
        end
end

# calculate the averages
pop_avg_array = mean(pop_array, dims=1)
pop_avg_array = dropdims(pop_avg_array, dims=1)

# calculate the std
pop_std_array = std(pop_array, dims=1)
pop_std_array = dropdims(pop_std_array, dims=1)

# Plot the figure

In [ ]:
using Plots
using LaTeXStrings

p = plot(title="Forward rate " * L"f=10^{-1}", xlabel="Time", ylabel="Count", yaxis=:log10, legendtitle="Integers", legend=:outerright)

palette = cgrad(:viridis, nspecies)

for i in 1:nspecies
    pop_avg = pop_avg_array[:, i]
    pop_std = pop_std_array[:, i]
    plot!(p, 1:max_t, pop_avg, lw=2, ribbon=pop_std, fillalpha=0.2, label="$i")
end

annotate!(75,700, text("Shading represents STD", :black, 10))

display(p)

savefig("./figs/pop_f=1e-1.pdf")